In [1]:
# KAGGLE/COLAB SETUP (uncomment on cloud)
import os
if not os.path.exists("llm-semeval-task8"):
  !git clone https://github.com/LookUpMark/llm-semeval-task8.git
%cd llm-semeval-task8
!git checkout dev
!pip install -q langchain langchain-community langchain-huggingface langchain-qdrant qdrant-client sentence-transformers bitsandbytes accelerate transformers tqdm langgraph

Cloning into 'llm-semeval-task8'...
remote: Enumerating objects: 641, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 641 (delta 165), reused 212 (delta 94), pack-reused 351 (from 1)
Receiving objects: 100% (641/641), 208.70 MiB | 22.49 MiB/s, done.
Resolving deltas: 100% (327/327), done.
/content/llm-semeval-task8
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
# @title 1. Installazione Librerie & Login Hugging Face
!pip install -q transformers bitsandbytes accelerate pandas scikit-learn tqdm sentencepiece
#!pip install ragas langchain-huggingface sentence-transformers

from google.colab import userdata
import os

# Se hai salvato il token nei segreti di Colab come 'HF_TOKEN', altrimenti inseriscilo manualmente sotto
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    HF_TOKEN = "INSERISCI_QUI_IL_TUO_TOKEN_HF_SE_USI_LLAMA3"

print("Librerie installate.")

Librerie installate.


In [3]:
# @title 2. Caricamento Modello Giudice (Quantizzato 4-bit)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Usiamo Llama-3-8B-Instruct. È eccellente come giudice.
# Se non hai accesso, cambia con "mistralai/Mistral-7B-Instruct-v0.2"
#MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"
MODEL_ID = "Qwen/Qwen2.5-14B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Caricamento {MODEL_ID}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
    tokenizer.pad_token_id = tokenizer.eos_token_id

    judge_model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        token=HF_TOKEN
    )
    print("Modello caricato con successo sulla T4!")
except Exception as e:
    print(f"ERRORE: {e}")
    print("Controlla il token HF o prova a cambiare MODEL_ID con 'mistralai/Mistral-7B-Instruct-v0.3'")

Caricamento Qwen/Qwen2.5-14B-Instruct...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Modello caricato con successo sulla T4!


In [ ]:
import json
import pandas as pd
from tqdm import tqdm
import torch
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM

# ==========================================
# 1. SETUP MODELLI
# ==========================================

print("Caricamento modello embeddings (all-MiniLM-L6-v2)...")
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Decommenta se necessario caricare il modello
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# judge_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

# ==========================================
# 2. FUNZIONI HELPER
# ==========================================

def get_embedding_similarity(text1, text2):
    if not text1 or not text2: return 0.0
    embeddings = embed_model.encode([str(text1), str(text2)])
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

def parse_scored_response(response_text, mode="faithfulness"):
    response_text = response_text.strip()

    # Estrazione Ragionamento
    reason_match = re.search(r"Reasoning:\s*(.*?)(?:\n|$|Score:|Hallucination:)", response_text, re.DOTALL | re.IGNORECASE)
    reasoning = reason_match.group(1).strip() if reason_match else response_text.replace("\n", " ")[:250]

    # Estrazione Score (1-10)
    score_match = re.search(r"Score:\s*\[?\[?(\d+(?:\.\d+)?)\]?\]?", response_text, re.IGNORECASE)
    try:
        score = float(score_match.group(1)) if score_match else 0.0
        score = max(1.0, min(10.0, score)) # Clamp tra 1 e 10
    except:
        score = 1.0 # Fallback pessimistico

    # Estrazione Hallucination
    is_hallucination = "N/A"
    if mode == "faithfulness":
        hallucination_match = re.search(r"Hallucination:\s*\[?\[?(YES|NO)\]?\]?", response_text, re.IGNORECASE)
        if hallucination_match:
            is_hallucination = hallucination_match.group(1).upper()

    return score, is_hallucination, reasoning

def ask_judge_cot(prompt):
    messages = [
        {"role": "system", "content": "You are a strict evaluator. Output strictly in the requested format."},
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(judge_model.device)
    terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

    with torch.no_grad():
        outputs = judge_model.generate(
            input_ids, max_new_tokens=150, eos_token_id=terminators, do_sample=False, temperature=0.0
        )
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True).strip()

# ==========================================
# 3. PROMPT ENGINEERING (Nuova Logica)
# ==========================================

def evaluate_faithfulness_scored(question, answer, context):
    """Caso Standard: Il modello ha risposto."""
    prompt = f"""Task: Evaluate the faithfulness of the Answer to the Context (1-10).

Context: {str(context)[:2500]}...
Answer: {str(answer)}

Instructions:
1. Identify if the Answer contains facts NOT present in the Context (Hallucinations).
2. Rate 1-4: Severe hallucinations.
3. Rate 5-7: Minor unverified details.
4. Rate 8-10: Fully supported.

Format:
Reasoning: [Explanation]
Hallucination: [[YES]] or [[NO]]
Score: [[1-10]]"""
    return parse_scored_response(ask_judge_cot(prompt), mode="faithfulness")

def evaluate_refusal_justification(question, context):
    """
    Caso I_DONT_KNOW: Valuta se il rifiuto era giustificato.
    Se il contesto HA la risposta, il punteggio deve essere BASSO (1).
    Se il contesto NON HA la risposta, il punteggio deve essere ALTO (10).
    """
    prompt = f"""Task: The model refused to answer ("I_DONT_KNOW"). Evaluate if this refusal was justified based on the Context.

Question: {question}
Context: {str(context)[:2500]}...

Instructions:
1. Analyze the Context to see if it contains the answer to the Question.
2. If the Context DOES contain the answer: The refusal is WRONG (False Negative). Score: 1.
3. If the Context does NOT contain the answer: The refusal is CORRECT. Score: 10.

Format:
Reasoning: [Did the context contain the info?]
Score: [[1-10]]"""

    # Usiamo 'correctness' come mode solo per ignorare il flag hallucination nel parsing
    score, _, reasoning = parse_scored_response(ask_judge_cot(prompt), mode="correctness")
    return score, reasoning

def evaluate_correctness_scored(question, gt, answer):
    """Caso Standard: Confronto semantico."""
    prompt = f"""Task: Rate semantic correctness (1-10).

Question: {question}
Ground Truth: {gt}
Generated Answer: {answer}

Instructions:
Rate 1-3: Wrong.
Rate 4-6: Partially correct.
Rate 7-10: Correct meaning.

Format:
Reasoning: [Explanation]
Score: [[1-10]]"""
    score, _, reasoning = parse_scored_response(ask_judge_cot(prompt), mode="correctness")
    return score, reasoning

# ==========================================
# 4. DATA LOADER & MAIN LOOP
# ==========================================

# CONFIGURAZIONE PATH
SUBMISSION_FILE = '/content/llm-semeval-task8/data/submissions/submission_TaskC_Gbgers.jsonl'
CONVERSATION_FILE = '/content/llm-semeval-task8/dataset/human/conversations/conversations.json'
OUTPUT_CSV = "evaluation_results_advanced.csv"

# Caricamento dati (invariato)
def load_data_optimized(conversations_path, submission_path):
    with open(conversations_path, 'r') as f: all_conversations = json.load(f)
    submission_data = []
    with open(submission_path, 'r') as f:
        for line in f:
            if line.strip(): submission_data.append(json.loads(line))

    aligned_data = []
    for item in submission_data:
        idx = item.get("original_index")
        if idx is None or idx >= len(all_conversations): continue
        conv = all_conversations[idx]
        msgs = conv.get("messages", [])
        last_user_msg = next((m["text"] for m in reversed(msgs) if m.get("speaker") == "user"), None)
        last_agent_msg = next((m["text"] for m in reversed(msgs) if m.get("speaker") != "user"), None)
        if not last_user_msg: continue

        refs = item.get("references", [])
        ctx_str = ""
        if isinstance(refs, list):
            for i, r in enumerate(refs):
                txt = r.get('text', str(r)) if isinstance(r, dict) else str(r)
                ctx_str += f"[DOC_{i+1}] {txt[:400]}...\n"

        aligned_data.append({
            'id': item.get('conversation_id', idx),
            'question': last_user_msg,
            'ground_truth': last_agent_msg if last_agent_msg else "I_DONT_KNOW",
            'generated_answer': item.get('answer', item.get('generated_answer', '')),
            'retrieved_context': ctx_str
        })
    return pd.DataFrame(aligned_data)

df = load_data_optimized(CONVERSATION_FILE, SUBMISSION_FILE)

headers = [
    'id', 'question', 'ground_truth', 'generated_answer', 'retrieved_context',
    'is_refusal', 'is_hallucination',
    'semantic_similarity',
    'faithfulness_score', 'correctness_score',
    'faithfulness_reason', 'correctness_reason'
]

pd.DataFrame(columns=headers).to_csv(OUTPUT_CSV, index=False)
results = []

print(f"\nAvvio Valutazione Avanzata ({len(df)} righe)...")

for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        q = row['question']
        gt = row['ground_truth']
        ans = row['generated_answer']
        ctx = row['retrieved_context']

        is_refusal = "I_DONT_KNOW" in ans
        sem_sim = get_embedding_similarity(ans, gt)

        # Init vars
        faith_score = 0.0
        corr_score = 0.0 # Sarà sovrascritto da float o "N/A"
        is_hallucination = "N/A"
        faith_reason = "N/A"
        corr_reason = "N/A"

        # --- LOGICA NUOVA ---
        if is_refusal:
            # CASO 1: IL MODELLO RIFIUTA
            # Correctness è N/A per definizione richiesta
            corr_score = "N/A"
            corr_reason = "Model Refused (Comparison N/A)"

            # Faithfulness: Valutiamo se ha fatto bene a rifiutare
            faith_score, faith_reason = evaluate_refusal_justification(q, ctx)

            # Hallucination è NO (non ha inventato nulla, ha rifiutato)
            is_hallucination = "NO"

        else:
            # CASO 2: IL MODELLO RISPONDE

            # 1. Faithfulness Standard (1-10) + Hallucination Check
            faith_score, is_hallucination, faith_reason = evaluate_faithfulness_scored(q, ans, ctx)

            # 2. Correctness Standard (1-10)
            if sem_sim > 0.95:
                corr_score = 10.0
                corr_reason = f"High Semantic Match ({sem_sim:.4f}) - Auto Score"
            else:
                corr_score, corr_reason = evaluate_correctness_scored(q, gt, ans)

        row_result = {
            'id': row['id'],
            'question': q,
            'ground_truth': gt,
            'generated_answer': ans,
            'retrieved_context': ctx,
            'is_refusal': is_refusal,
            'is_hallucination': is_hallucination,
            'semantic_similarity': round(sem_sim, 4),
            'faithfulness_score': faith_score,
            'correctness_score': corr_score,
            'faithfulness_reason': faith_reason,
            'correctness_reason': corr_reason
        }

        results.append(row_result)
        pd.DataFrame([row_result]).to_csv(OUTPUT_CSV, mode='a', header=False, index=False)

    except Exception as e:
        print(f"Errore riga {index}: {e}")
        err_row = {k: "ERROR" for k in headers}
        err_row['id'] = row.get('id', index)
        pd.DataFrame([err_row]).to_csv(OUTPUT_CSV, mode='a', header=False, index=False)

print(f"\nFatto! File salvato: {OUTPUT_CSV}")

# Mini Report (escludendo N/A per le medie)
res_df = pd.DataFrame(results)
if not res_df.empty:
    # Convertiamo in numeric, coercing errors ('N/A') to NaN
    faith_numeric = pd.to_numeric(res_df['faithfulness_score'], errors='coerce')
    corr_numeric = pd.to_numeric(res_df['correctness_score'], errors='coerce')

    print("\nREPORT STATISTICO:")
    print(f"Avg Faithfulness (Refusals included): {faith_numeric.mean():.2f}")
    print(f"Avg Correctness (Answers only): {corr_numeric.mean():.2f}")
    print(f"Refusal Rate: {res_df['is_refusal'].mean():.2%}")

Caricamento modello embeddings (all-MiniLM-L6-v2)...

Avvio Valutazione Avanzata (110 righe)...


  1%|          | 1/110 [00:00<00:20,  5.39it/s]

Errore riga 0: 'NoneType' object cannot be interpreted as an integer


  6%|▋         | 7/110 [00:00<00:03, 29.03it/s]

Errore riga 1: 'NoneType' object cannot be interpreted as an integer
Errore riga 2: 'NoneType' object cannot be interpreted as an integer
Errore riga 3: 'NoneType' object cannot be interpreted as an integer
Errore riga 4: 'NoneType' object cannot be interpreted as an integer
Errore riga 5: 'NoneType' object cannot be interpreted as an integer
Errore riga 6: 'NoneType' object cannot be interpreted as an integer
Errore riga 7: 'NoneType' object cannot be interpreted as an integer
Errore riga 8: 'NoneType' object cannot be interpreted as an integer
Errore riga 9: 'NoneType' object cannot be interpreted as an integer
Errore riga 10: 'NoneType' object cannot be interpreted as an integer
Errore riga 11: 'NoneType' object cannot be interpreted as an integer
Errore riga 12: 'NoneType' object cannot be interpreted as an integer
Errore riga 13: 'NoneType' object cannot be interpreted as an integer


...

Errore riga 77: 'NoneType' object cannot be interpreted as an integer
Errore riga 78: 'NoneType' object cannot be interpreted as an integer
Errore riga 79: 'NoneType' object cannot be interpreted as an integer
Errore riga 80: 'NoneType' object cannot be interpreted as an integer
Errore riga 81: 'NoneType' object cannot be interpreted as an integer
Errore riga 82: 'NoneType' object cannot be interpreted as an integer
Errore riga 83: 'NoneType' object cannot be interpreted as an integer
Errore riga 84: 'NoneType' object cannot be interpreted as an integer
Errore riga 85: 'NoneType' object cannot be interpreted as an integer
Errore riga 86: 'NoneType' object cannot be interpreted as an integer
Errore riga 87: 'NoneType' object cannot be interpreted as an integer
Errore riga 88: 'NoneType' object cannot be interpreted as an integer
Errore riga 89: 'NoneType' object cannot be interpreted as an integer
Errore riga 90: 'NoneType' object cannot be interpreted as an integer
Errore riga 91: 'Non

 94%|█████████▎| 103/110 [00:01<00:00, 72.41it/s]

Errore riga 92: 'NoneType' object cannot be interpreted as an integer
Errore riga 93: 'NoneType' object cannot be interpreted as an integer
Errore riga 94: 'NoneType' object cannot be interpreted as an integer
Errore riga 95: 'NoneType' object cannot be interpreted as an integer
Errore riga 96: 'NoneType' object cannot be interpreted as an integer
Errore riga 97: 'NoneType' object cannot be interpreted as an integer
Errore riga 98: 'NoneType' object cannot be interpreted as an integer
Errore riga 99: 'NoneType' object cannot be interpreted as an integer
Errore riga 100: 'NoneType' object cannot be interpreted as an integer
Errore riga 101: 'NoneType' object cannot be interpreted as an integer
Errore riga 102: 'NoneType' object cannot be interpreted as an integer
Errore riga 103: 'NoneType' object cannot be interpreted as an integer
Errore riga 104: 'NoneType' object cannot be interpreted as an integer
Errore riga 105: 'NoneType' object cannot be interpreted as an integer
Errore riga 10

100%|██████████| 110/110 [00:01<00:00, 65.09it/s]

Errore riga 107: 'NoneType' object cannot be interpreted as an integer
Errore riga 108: 'NoneType' object cannot be interpreted as an integer
Errore riga 109: 'NoneType' object cannot be interpreted as an integer

Fatto! File salvato: evaluation_results_advanced.csv


In [ ]:
# Qwen
import json
import pandas as pd
from tqdm import tqdm
import torch
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM

# ==========================================
# 1. SETUP MODELLI
# ==========================================

print("Caricamento modello embeddings (all-MiniLM-L6-v2)...")
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Configurazione per Qwen/Qwen2.5-14B-Instruct
# Nota: Assicurati di avere abbastanza VRAM. Se usi Colab T4, potresti dover usare la versione quantizzata (AWQ o GPTQ)
# oppure caricare in 8bit/4bit usando bitsandbytes.
model_id = "Qwen/Qwen2.5-14B-Instruct"

print(f"Caricamento LLM Giudice: {model_id}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
except Exception as e:
    print(f"Errore nel caricamento del modello (potrebbe essere memoria insufficiente): {e}")
    # Fallback per test o se il modello è già caricato esternamente
    pass

# ==========================================
# 2. FUNZIONI HELPER
# ==========================================

def get_embedding_similarity(text1, text2):
    if not text1 or not text2: return 0.0
    embeddings = embed_model.encode([str(text1), str(text2)])
    return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

def parse_scored_response(response_text, mode="faithfulness"):
    """Estrae Reasoning, Score (1-10) e Hallucination (YES/NO) usando Regex."""
    response_text = response_text.strip()

    # 1. Estrazione Ragionamento
    # Cerca testo dopo 'Reasoning:' fino a 'Score:', 'Hallucination:' o fine riga
    reason_match = re.search(r"Reasoning:\s*(.*?)(?:\n|$|Score:|Hallucination:)", response_text, re.DOTALL | re.IGNORECASE)
    reasoning = reason_match.group(1).strip() if reason_match else response_text.replace("\n", " ")[:250]

    # 2. Estrazione Score (Numeri float o int tra 1 e 10)
    score_match = re.search(r"Score:\s*\[?\[?(\d+(?:\.\d+)?)\]?\]?", response_text, re.IGNORECASE)
    try:
        score = float(score_match.group(1)) if score_match else 0.0
        score = max(1.0, min(10.0, score)) # Clamp tra 1 e 10
    except:
        score = 1.0 # Fallback pessimistico se il parsing fallisce

    # 3. Estrazione Hallucination Flag
    is_hallucination = "N/A"
    if mode == "faithfulness":
        hallucination_match = re.search(r"Hallucination:\s*\[?\[?(YES|NO)\]?\]?", response_text, re.IGNORECASE)
        if hallucination_match:
            is_hallucination = hallucination_match.group(1).upper()

    return score, is_hallucination, reasoning

def ask_judge_cot(prompt):
    """Genera la risposta gestendo correttamente i token di stop per Qwen."""
    messages = [
        {"role": "system", "content": "You are a strict AI evaluator. Analyze the text objectively and adhere strictly to the requested output format (JSON-like keys: Reasoning, Score, Hallucination)."},
        {"role": "user", "content": prompt}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(judge_model.device)

    # --- FIX CRITICO PER QWEN ---
    # Costruiamo la lista terminators in modo sicuro.
    terminators = [tokenizer.eos_token_id]

    # Qwen usa spesso <|im_end|> (id 151645 o simile)
    # Llama usa <|eot_id|>
    # Controlliamo se esistono nel vocabolario prima di aggiungerli
    potential_stops = ["<|im_end|>", "<|endoftext|>", "<|eot_id|>"]

    for token_str in potential_stops:
        token_id = tokenizer.convert_tokens_to_ids(token_str)
        # tokenizer.convert_tokens_to_ids ritorna None se il token non esiste
        if token_id is not None:
            terminators.append(token_id)
    # ----------------------------

    with torch.no_grad():
        outputs = judge_model.generate(
            input_ids,
            max_new_tokens=200, # Aumentato leggermente per permettere ragionamenti più complessi
            eos_token_id=terminators,
            do_sample=False, # Determinismo massimo
            temperature=0.0
        )

    # Decodifica saltando i token speciali
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True).strip()

# ==========================================
# 3. PROMPT ENGINEERING (Logica Completa)
# ==========================================

def evaluate_faithfulness_scored(question, answer, context):
    """Standard Faithfulness: Verifica allucinazioni nella risposta data."""
    prompt = f"""Task: Evaluate the faithfulness of the Answer to the Context (Score 1-10).

Context: {str(context)[:2500]}...
Answer: {str(answer)}

Instructions:
1. Identify if the Answer contains facts NOT present in the Context (Hallucinations).
2. Rate 1-4: Severe hallucinations (invented facts).
3. Rate 5-7: Minor unverified details.
4. Rate 8-10: Fully supported by the provided text.

Required Format:
Reasoning: [Brief explanation]
Hallucination: [[YES]] or [[NO]]
Score: [[1-10]]"""
    return parse_scored_response(ask_judge_cot(prompt), mode="faithfulness")

def evaluate_refusal_justification(question, context):
    """
    Caso I_DONT_KNOW: Valuta se il rifiuto era giustificato.
    - Se il contesto CONTIENE la risposta -> Rifiuto ingiustificato (Score 1).
    - Se il contesto NON contiene la risposta -> Rifiuto corretto (Score 10).
    """
    prompt = f"""Task: The model refused to answer ("I_DONT_KNOW"). Evaluate if this refusal was correct based on the Context.

Question: {question}
Context: {str(context)[:2500]}...

Instructions:
1. Check if the Context contains the information needed to answer the Question.
2. If Context HAS the answer: The refusal is a False Negative (Bad). Score: 1.
3. If Context does NOT have the answer: The refusal is Correct (Good). Score: 10.

Required Format:
Reasoning: [Did the context contain the info?]
Score: [[1-10]]"""

    # Usiamo mode="correctness" per ignorare il campo Hallucination
    score, _, reasoning = parse_scored_response(ask_judge_cot(prompt), mode="correctness")
    return score, reasoning

def evaluate_correctness_scored(question, gt, answer):
    """Standard Correctness: Confronto semantico con Ground Truth."""
    prompt = f"""Task: Rate the semantic correctness of the Answer vs Ground Truth (Score 1-10).

Question: {question}
Ground Truth: {gt}
Generated Answer: {answer}

Instructions:
Rate 1-3: Completely wrong.
Rate 4-6: Partially correct but misses key points.
Rate 7-10: Correct meaning (even if phrasing differs).

Required Format:
Reasoning: [Brief explanation]
Score: [[1-10]]"""
    score, _, reasoning = parse_scored_response(ask_judge_cot(prompt), mode="correctness")
    return score, reasoning

# ==========================================
# 4. DATA LOADER & MAIN LOOP
# ==========================================

SUBMISSION_FILE = '/content/llm-semeval-task8/data/submissions/submission_TaskC_Gbgers.jsonl'
CONVERSATION_FILE = '/content/llm-semeval-task8/dataset/human/conversations/conversations.json'
OUTPUT_CSV = "evaluation_results_qwen_14b.csv"

def load_data_optimized(conversations_path, submission_path):
    print("Caricamento Dataset...")
    with open(conversations_path, 'r') as f: all_conversations = json.load(f)
    submission_data = []
    with open(submission_path, 'r') as f:
        for line in f:
            if line.strip(): submission_data.append(json.loads(line))

    aligned_data = []
    for item in submission_data:
        idx = item.get("original_index")
        if idx is None or idx >= len(all_conversations): continue
        conv = all_conversations[idx]
        msgs = conv.get("messages", [])
        last_user_msg = next((m["text"] for m in reversed(msgs) if m.get("speaker") == "user"), None)
        last_agent_msg = next((m["text"] for m in reversed(msgs) if m.get("speaker") != "user"), None)
        if not last_user_msg: continue

        # Formattazione Contesto per CSV e Prompt
        refs = item.get("references", [])
        ctx_str = ""
        if isinstance(refs, list):
            for i, r in enumerate(refs):
                txt = r.get('text', str(r)) if isinstance(r, dict) else str(r)
                ctx_str += f"[DOC_{i+1}] {txt[:400]}...\n" # Tronchiamo per leggibilità

        aligned_data.append({
            'id': item.get('conversation_id', idx),
            'question': last_user_msg,
            'ground_truth': last_agent_msg if last_agent_msg else "I_DONT_KNOW",
            'generated_answer': item.get('answer', item.get('generated_answer', '')),
            'retrieved_context': ctx_str
        })
    return pd.DataFrame(aligned_data)

# Caricamento dati
df = load_data_optimized(CONVERSATION_FILE, SUBMISSION_FILE)

# Headers del CSV finale
headers = [
    'id', 'question', 'ground_truth', 'generated_answer', 'retrieved_context',
    'is_refusal', 'is_hallucination',
    'semantic_similarity',
    'faithfulness_score', 'correctness_score',
    'faithfulness_reason', 'correctness_reason'
]

# Inizializza file CSV
pd.DataFrame(columns=headers).to_csv(OUTPUT_CSV, index=False)
results = []

print(f"\nAvvio Valutazione con Qwen ({len(df)} righe)...")

for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        q = row['question']
        gt = row['ground_truth']
        ans = row['generated_answer']
        ctx = row['retrieved_context']

        is_refusal = "I_DONT_KNOW" in ans
        sem_sim = get_embedding_similarity(ans, gt)

        # Inizializzazione variabili output
        faith_score = 0.0
        corr_score = 0.0 # Rimarrà 0.0 o diventerà "N/A"
        is_hallucination = "N/A"
        faith_reason = "N/A"
        corr_reason = "N/A"

        # --- LOGICA DI VALUTAZIONE ---
        if is_refusal:
            # === CASO 1: IL MODELLO HA RIFIUTATO ===

            # Correctness: Non applicabile secondo specifica
            corr_score = "N/A"
            corr_reason = "Model Refused (Comparison N/A)"

            # Faithfulness: Giudichiamo se il rifiuto era giustificato dal contesto
            faith_score, faith_reason = evaluate_refusal_justification(q, ctx)

            # Hallucination: NO (non ha inventato nulla)
            is_hallucination = "NO"

        else:
            # === CASO 2: IL MODELLO HA RISPOSTO ===

            # 1. Faithfulness Check (1-10 + Hallucination)
            faith_score, is_hallucination, faith_reason = evaluate_faithfulness_scored(q, ans, ctx)

            # 2. Correctness Check (1-10)
            if sem_sim > 0.95:
                # Scorciatoia se embedding molto simili
                corr_score = 10.0
                corr_reason = f"High Semantic Match ({sem_sim:.4f}) - Auto Score"
            else:
                corr_score, corr_reason = evaluate_correctness_scored(q, gt, ans)

        # Salvataggio Risultati
        row_result = {
            'id': row['id'],
            'question': q,
            'ground_truth': gt,
            'generated_answer': ans,
            'retrieved_context': ctx,
            'is_refusal': is_refusal,
            'is_hallucination': is_hallucination,
            'semantic_similarity': round(sem_sim, 4),
            'faithfulness_score': faith_score,
            'correctness_score': corr_score,
            'faithfulness_reason': faith_reason,
            'correctness_reason': corr_reason
        }

        results.append(row_result)
        pd.DataFrame([row_result]).to_csv(OUTPUT_CSV, mode='a', header=False, index=False)

    except Exception as e:
        print(f"Errore riga {index}: {e}")
        err_row = {k: "ERROR" for k in headers}
        err_row['id'] = row.get('id', index)
        pd.DataFrame([err_row]).to_csv(OUTPUT_CSV, mode='a', header=False, index=False)

print(f"\nValutazione completata! File salvato: {OUTPUT_CSV}")

# Mini Report Finale
res_df = pd.DataFrame(results)
if not res_df.empty:
    # Convertiamo colonne numeriche gestendo gli "N/A"
    faith_numeric = pd.to_numeric(res_df['faithfulness_score'], errors='coerce')
    corr_numeric = pd.to_numeric(res_df['correctness_score'], errors='coerce')

    print("\n=== REPORT FINALE ===")
    print(f"Avg Faithfulness (Tutti i casi): {faith_numeric.mean():.2f}/10")
    print(f"Avg Correctness (Solo risposte date): {corr_numeric.mean():.2f}/10")
    print(f"Tasso di Rifiuto (I_DONT_KNOW): {res_df['is_refusal'].mean():.2%}")

    hallucinations = res_df[res_df['is_hallucination'] == 'YES'].shape[0]
    print(f"Allucinazioni rilevate (sulle risposte date): {hallucinations}")

Caricamento modello embeddings (all-MiniLM-L6-v2)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Caricamento LLM Giudice: Qwen/Qwen2.5-14B-Instruct...
Caricamento Dataset...

Avvio Valutazione con Qwen (110 righe)...


  0%|          | 0/110 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 110/110 [42:32<00:00, 23.20s/it]


Valutazione completata! File salvato: evaluation_results_qwen_14b.csv

=== REPORT FINALE ===
Avg Faithfulness (Tutti i casi): 3.46/10
Avg Correctness (Solo risposte date): 2.16/10
Tasso di Rifiuto (I_DONT_KNOW): 36.36%
Allucinazioni rilevate (sulle risposte date): 61
